In [28]:
import pandas as pd
import numpy as np

from collections import defaultdict

In [29]:
def get_similarity_matrix(source, preference_func):
    func = lambda i, j: -np.sum(np.square(source[i] - source[j]))
    similiarity_matrix = np.fromfunction(np.vectorize(func), (source.shape[0], source.shape[0]), dtype=int)
    np.fill_diagonal(similiarity_matrix, preference_func(similiarity_matrix))
    return similiarity_matrix

def responsibility(s, a, i, k, mask):
    mask[k] = False
    result = s[i][k] - np.amax(a[i][mask] + s[i][mask])
    mask[k] = True
    return result

def availability(i, k, r, mask):
    mask[k] = False
    mask[i] = False
    column = r[:, k] * mask
    result = np.sum(column[column > 0]) if i == k else min(0, r[k][k] + np.sum(column[column > 0]))
    mask[k] = True
    mask[i] = True
    return result

def run_affinity_propagation(source_matrix, preference_func, iterations=10):
    s = get_similarity_matrix(source_matrix, preference_func)
    a = np.zeros((s.shape[0], s.shape[0]), dtype=int)
    c = np.zeros((s.shape[0], s.shape[0]), dtype=int)

    mask = np.full(s.shape[0], True, dtype=bool)

    for iteration in range(iterations):
        print(f"Iteration {iteration + 1}")
        r = np.fromfunction(
            np.vectorize(lambda i, k: responsibility(s, a, i, k, mask)),
            (s.shape[0], s.shape[0]),
            dtype=int
        )
        a = np.fromfunction(
            np.vectorize(lambda i, k: availability(i, k, r, mask)),
            (r.shape[0], r.shape[0]),
            dtype=int
        )
        c = r + a
    return np.argmax(c, axis=1)

In [32]:
adjacency = np.zeros((1000, 1000), dtype=int)

with open("Gowalla_edges.txt") as network_file:
    for line in network_file:
        line = line.strip()
        a, b = map(int, line.split())

        if a < 1000 and b < 1000:
            adjacency[a][b] = 1
            adjacency[b][a] = 1

In [33]:
propagation_results = run_affinity_propagation(adjacency, iterations=1, preference_func=np.min)
number_of_clusters = len(np.unique(propagation_results))
print(f"Total number of clusters is {number_of_clusters}")
clusters = defaultdict(list)
for i, entry in enumerate(propagation_results):
    clusters[entry].append(i)
checkins = pd.read_csv("Gowalla_totalCheckins.txt", delimiter = "\t", index_col=False, names = ["User", "Time", "Latitude", "Longitude", "Place"])
checkins.head()

Iteration 1
Total number of clusters is 115


,User,Time,Latitude,Longitude,Place
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


In [ ]:
test_user_data = np.random.choice(1000, 100, replace = False)
average_percentage = 0
for test_user in test_user_data:
    user_cluster = propagation_results[test_user]
    user_places  = defaultdict(int)

    for user in clusters[user_cluster]:
        if test_user != user:
            for place in pd.unique(checkins[checkins["User"] == user]["Place"]):
                user_places[place] += 1

    top_places = sorted(user_places, key=user_places.get, reverse=True)[:10]

    matches = 0
    for place in top_places:
        if place in checkins[checkins["User"] == test_user]["Place"]:
            matches += 1

    user_percentage = matches * 10  # (matches / 10) * 100
    average_percentage += user_percentage

average_percentage /= len(test_user_data)
print(f"Average match percentage: {average_percentage}%")